In [1]:
import pandas as pd
import numpy as np

In [2]:
data_dir = '../data/train_test/preprocessed'
selected_bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12']

## Data preprocessing

### Augmenting rows

In [3]:
def add_column_statistics(df, columns, by_column=None, add_std=False):
    df = df.copy()

    if by_column:
        for i in columns:
            df[f'{i}_min_by_fid'] = df.groupby(by_column)[i].transform('min')
            df[f'{i}_max_by_fid'] = df.groupby(by_column)[i].transform('max')
            df[f'{i}_avg_by_fid'] = df.groupby(by_column)[i].transform('mean')

            if add_std:
                df[f'{i}_std_by_fid'] = df.groupby(by_column)[i].transform('std')
    else:
        for i in columns:
            if add_std:
                df[f'{i}_std'] = df.filter(regex = f'^{i}').std(axis = 1)
                
            df[f'{i}_max'] = df.filter(regex = f'^{i}').max(axis = 1)
            df[f'{i}_min'] = df.filter(regex = f'^{i}').min(axis = 1)
            df[f'{i}_avg'] =df.filter(regex = f'^{i}').mean(axis = 1)

    return df
    

def add_column_mappings(df, columns):
    df = df.copy()

    for i in columns:
        df[f'{i}_sqrt'] = np.sqrt(df[i])
        df[f'{i}_exp'] = np.exp(df[i])
        df[f'{i}_^2'] = df[i]**2
    
    return df

### Add spetral indices

In [4]:
spectral_indices = [
    "NDVI",
    "GNDVI",
    # "EVI",
    "EVI2",
    # "AVI",
    "BSI",
    "SI",
    "NDWI",
    "NDMI",
    "NPCRI",
    "SAVI",
    "MSI",
    "GCI",
    "NBRI",
    "NDSI",
    "NDGI",
    "ARVI",
    "SIPI",
]

def add_spectral_indices(df):
    df = df.copy()
    
    df["NDVI"] = (df["B08"] - df["B04"]) / (df["B08"] + df["B04"])
    df["GNDVI"] = (df["B08"] - df["B03"]) / (df["B08"] + df["B03"])
    # df["EVI"] = 2.5 * ((df["B08"] - df["B04"]) / ((df["B08"] + 6.0 * df["B04"] - 7.5 * df['B02']) + 1.0))
    df["EVI2"] = 2.4 * (df["B08"] - df["B04"]) / (df["B08"] + df["B04"] + 1.0)
    # df["AVI"] = (df["B08"] * (1 - df["B04"]) * (df["B08"] - df["B04"]))**(1/3)
    df["BSI"] = ((df["B11"] + df["B04"]) - (df["B08"] + df["B02"])) / ((df["B11"] + df["B04"]) + (df["B08"] + df["B02"]))
    df["SI"] = ((1 - df["B02"]) * (1 - df["B03"]) * (1 - df["B04"]))
    df["NDWI"] = (df["B03"] - df["B08"]) / (df["B03"] + df["B08"])
    df["NDMI"] = (df["B08"] - df["B11"]) / (df["B08"] + df["B11"]) 
    df["NPCRI"] = (df["B04"] - df["B02"]) / (df["B04"] + df["B02"]) 
    df["SAVI"] = (df["B08"] - df["B04"]) / (df["B08"] + df["B04"] + 0.428) * (1.428)
    df["MSI"] = df["B11"] / df["B08"]
    df["GCI"] = (df["B09"] / df["B03"]) - 1
    df["NBRI"] = (df["B08"] - df["B12"]) / (df["B08"] + df["B12"])
    df["NDSI"] = (df["B03"] - df["B11"]) / (df["B03"] + df["B11"])
    df["NDGI"] = (df["B03"] - df["B04"]) / (df["B03"] + df["B04"]) 
    df["ARVI"] = (df["B08"] - (2 * df["B04"]) + df["B02"]) / (df["B08"] + (2 * df["B04"]) + df["B02"]) 
    df["SIPI"] = (df["B08"] - df["B02"]) / (df["B08"] - df["B04"])

    return df

In [5]:
df_train = pd.read_csv(f"{data_dir}/tabular_train.csv")
df_test = pd.read_csv(f"{data_dir}/tabular_test.csv")
df_all = pd.concat([df_train, df_test])

display(df_all.head())

,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,B11,B12,field_id,crop
0,44,40,40,42,45,59,70,63,76,13,78,55,1374,1.0
1,44,40,40,42,45,59,70,62,76,13,78,55,1374,1.0
2,44,40,39,42,46,52,58,54,62,16,72,53,3293,1.0
3,44,41,39,43,48,58,65,61,71,16,78,61,3293,1.0
4,44,40,40,44,48,58,65,64,71,16,78,61,3293,1.0


In [6]:
# spectral bands
df_all = add_column_statistics(df_all, selected_bands)
df_all = add_column_mappings(df_all, selected_bands)
df_all = add_column_statistics(df_all, selected_bands, "field_id")

# spectral indices
df_all = add_spectral_indices(df_all)
df_all = add_column_statistics(df_all, spectral_indices)
df_all = add_column_mappings(df_all, spectral_indices)
df_all = add_column_statistics(df_all, spectral_indices, "field_id")

display(df_all.head())

/home/arnol/anaconda3/envs/aic/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/arnol/anaconda3/envs/aic/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/arnol/anaconda3/envs/aic/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/arnol/anaconda3/envs/aic/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/arnol/anaconda3/envs/aic/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/arnol/anaconda3/envs/aic/lib/

,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,...,NDSI_avg_by_fid,NDGI_min_by_fid,NDGI_max_by_fid,NDGI_avg_by_fid,ARVI_min_by_fid,ARVI_max_by_fid,ARVI_avg_by_fid,SIPI_min_by_fid,SIPI_max_by_fid,SIPI_avg_by_fid
0,44,40,40,42,45,59,70,63,76,13,...,-0.317357,-0.057471,0.0,-0.032736,0.065990,0.146067,0.091702,1.000000,1.277778,1.139093
1,44,40,40,42,45,59,70,62,76,13,...,-0.317357,-0.057471,0.0,-0.032736,0.065990,0.146067,0.091702,1.000000,1.277778,1.139093
2,44,40,39,42,46,52,58,54,62,16,...,-0.312590,-0.085106,0.0,-0.051285,0.042254,0.139535,0.073813,0.909091,1.500000,1.232843
3,44,41,39,43,48,58,65,61,71,16,...,-0.312590,-0.085106,0.0,-0.051285,0.042254,0.139535,0.073813,0.909091,1.500000,1.232843
4,44,40,40,44,48,58,65,64,71,16,...,-0.312590,-0.085106,0.0,-0.051285,0.042254,0.139535,0.073813,0.909091,1.500000,1.232843


In [7]:
df_all = df_all.join(df_all.groupby("field_id").size().rename(f'num_pixels_by_fid'), on='field_id')

df_all.head()

,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,...,NDGI_min_by_fid,NDGI_max_by_fid,NDGI_avg_by_fid,ARVI_min_by_fid,ARVI_max_by_fid,ARVI_avg_by_fid,SIPI_min_by_fid,SIPI_max_by_fid,SIPI_avg_by_fid,num_pixels_by_fid
0,44,40,40,42,45,59,70,63,76,13,...,-0.057471,0.0,-0.032736,0.065990,0.146067,0.091702,1.000000,1.277778,1.139093,36
1,44,40,40,42,45,59,70,62,76,13,...,-0.057471,0.0,-0.032736,0.065990,0.146067,0.091702,1.000000,1.277778,1.139093,36
2,44,40,39,42,46,52,58,54,62,16,...,-0.085106,0.0,-0.051285,0.042254,0.139535,0.073813,0.909091,1.500000,1.232843,19
3,44,41,39,43,48,58,65,61,71,16,...,-0.085106,0.0,-0.051285,0.042254,0.139535,0.073813,0.909091,1.500000,1.232843,19
4,44,40,40,44,48,58,65,64,71,16,...,-0.085106,0.0,-0.051285,0.042254,0.139535,0.073813,0.909091,1.500000,1.232843,19


In [8]:
tmp = df_all.drop(columns=["crop"])

tmp.loc[:, tmp.isna().any()]

,GNDVI_sqrt,BSI_sqrt,SI_sqrt,NDWI_sqrt,NDMI_sqrt,NPCRI_sqrt,GCI_sqrt,NBRI_sqrt,NDSI_sqrt,NDGI_sqrt,ARVI_sqrt,SIPI_sqrt
0,0.472547,0.276104,NaN,NaN,NaN,0.156174,NaN,0.260378,NaN,NaN,0.318754,1.046536
1,0.464420,0.284747,NaN,NaN,NaN,0.156174,NaN,0.244600,NaN,NaN,0.311086,1.048809
2,0.401610,0.310087,NaN,NaN,NaN,0.156174,NaN,0.096674,NaN,NaN,0.237023,1.080123
3,0.469042,0.291893,NaN,NaN,NaN,0.154303,NaN,0.000000,NaN,NaN,0.291730,1.054093
4,0.480384,0.282216,NaN,NaN,NaN,0.218218,NaN,0.154919,NaN,NaN,0.288675,1.095445
...,...,...,...,...,...,...,...,...,...,...,...,...
49313,0.396059,0.337691,NaN,NaN,NaN,0.311086,NaN,0.187317,NaN,NaN,NaN,1.457738
49314,0.365148,0.349215,NaN,NaN,NaN,0.267261,NaN,NaN,NaN,NaN,0.000000,1.414214
49315,0.351562,0.350931,NaN,NaN,NaN,0.245440,NaN,NaN,NaN,NaN,0.075165,1.354006
49316,0.385922,0.325222,NaN,NaN,NaN,0.267261,NaN,0.137361,NaN,NaN,0.128037,1.290994


## Feature selection

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
df_all = df_all.reset_index(drop=True)
df_all.head()

,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,...,NDGI_min_by_fid,NDGI_max_by_fid,NDGI_avg_by_fid,ARVI_min_by_fid,ARVI_max_by_fid,ARVI_avg_by_fid,SIPI_min_by_fid,SIPI_max_by_fid,SIPI_avg_by_fid,num_pixels_by_fid
0,44,40,40,42,45,59,70,63,76,13,...,-0.057471,0.0,-0.032736,0.065990,0.146067,0.091702,1.000000,1.277778,1.139093,36
1,44,40,40,42,45,59,70,62,76,13,...,-0.057471,0.0,-0.032736,0.065990,0.146067,0.091702,1.000000,1.277778,1.139093,36
2,44,40,39,42,46,52,58,54,62,16,...,-0.085106,0.0,-0.051285,0.042254,0.139535,0.073813,0.909091,1.500000,1.232843,19
3,44,41,39,43,48,58,65,61,71,16,...,-0.085106,0.0,-0.051285,0.042254,0.139535,0.073813,0.909091,1.500000,1.232843,19
4,44,40,40,44,48,58,65,64,71,16,...,-0.085106,0.0,-0.051285,0.042254,0.139535,0.073813,0.909091,1.500000,1.232843,19


In [11]:
X_train = df_all[~df_all["crop"].isnull()]
X_test = df_all[df_all["crop"].isnull()]
y_train = X_train["crop"]

X_train = X_train.drop(columns=["field_id", "crop"])
X_test = X_test.drop(columns=["crop"])

In [12]:
X_train.shape, y_train.shape, X_test.shape

((188228, 281), (188228,), (49318, 282))

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, stratify = y_train, random_state = 5, shuffle = True)

In [14]:
import catboost
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_predict
from sklearn.utils.class_weight import compute_class_weight

In [15]:
label_weights = compute_class_weight("balanced", classes=np.unique(y_train), y=y_train)

In [16]:
# cb_pi --> catboost_permutation_importance\n",
cb_pi = CatBoostClassifier(n_estimators = 1400, learning_rate = 0.03, random_state = 11, task_type = "GPU")
cb_pi.fit(X_train, y_train)

0:	learn: 2.4195451	total: 54.1ms	remaining: 1m 15s
1:	learn: 2.3005404	total: 100ms	remaining: 1m 9s
2:	learn: 2.2023686	total: 144ms	remaining: 1m 7s
3:	learn: 2.1196715	total: 187ms	remaining: 1m 5s
4:	learn: 2.0476011	total: 234ms	remaining: 1m 5s
5:	learn: 1.9827399	total: 276ms	remaining: 1m 4s
6:	learn: 1.9247837	total: 319ms	remaining: 1m 3s
7:	learn: 1.8719587	total: 364ms	remaining: 1m 3s
8:	learn: 1.8230918	total: 406ms	remaining: 1m 2s
9:	learn: 1.7783691	total: 446ms	remaining: 1m 2s
10:	learn: 1.7370596	total: 488ms	remaining: 1m 1s
11:	learn: 1.7001469	total: 532ms	remaining: 1m 1s
12:	learn: 1.6654703	total: 574ms	remaining: 1m 1s
13:	learn: 1.6320735	total: 616ms	remaining: 1m
14:	learn: 1.6015821	total: 661ms	remaining: 1m 1s
15:	learn: 1.5733005	total: 704ms	remaining: 1m
16:	learn: 1.5459479	total: 746ms	remaining: 1m
17:	learn: 1.5207190	total: 786ms	remaining: 1m
18:	learn: 1.4960027	total: 830ms	remaining: 1m
19:	learn: 1.4739010	total: 867ms	remaining: 59.9s
20:

In [17]:
import eli5
from eli5.sklearn import PermutationImportance

In [18]:
pi = PermutationImportance(cb_pi, random_state = 90, n_iter = 5)
pi.fit(X_val, y_val)
eli5.show_weights(pi, feature_names = X_train.columns.tolist(), top = None)

Weight,Feature
0.0482 ± 0.0013,num_pixels_by_fid
0.0402 ± 0.0011,NDSI_min_by_fid
0.0272 ± 0.0016,NDGI_avg_by_fid
0.0258 ± 0.0006,NDGI_min_by_fid
0.0168 ± 0.0012,NBRI_min_by_fid
0.0116 ± 0.0003,NBRI_avg_by_fid
0.0090 ± 0.0008,NDSI_avg_by_fid
0.0084 ± 0.0004,B12_max_by_fid
0.0082 ± 0.0007,B02_avg_by_fid
0.0081 ± 0.0004,GCI_avg_by_fid


In [19]:
pi_results = eli5.formatters.as_dataframe.explain_weights_df(pi, feature_names = X_train.columns.tolist())
# feature importance weigth threshold is 0\n",
low_importance = pi_results[pi_results.weight <= 0].feature.values
low_importance

array(['NDMI_^2', 'B06_exp', 'B8A_sqrt', 'B08_^2', 'B08_exp', 'B08_sqrt',
       'B07_^2', 'B07_exp', 'B07_sqrt', 'B06_^2', 'B06_sqrt', 'B09_sqrt',
       'B05_^2', 'B05_exp', 'B05_sqrt', 'B04_^2', 'B04_exp', 'B04_sqrt',
       'B03_^2', 'B03_exp', 'B03_sqrt', 'B8A_^2', 'B09_exp', 'B02_exp',
       'ARVI_^2', 'GCI_^2', 'NBRI_sqrt', 'NBRI_exp', 'NBRI_^2',
       'NDSI_sqrt', 'NDMI_exp', 'ARVI_sqrt', 'ARVI_exp', 'SIPI_sqrt',
       'B09_^2', 'SIPI_exp', 'SIPI_^2', 'B12_^2', 'B12_exp', 'B12_sqrt',
       'B11_^2', 'B11_exp', 'B11_sqrt', 'B02_^2', 'B02_sqrt', 'MSI_^2',
       'B05_max', 'B04_min', 'B04_max', 'B03_avg', 'B03_min', 'B03_max',
       'B02_avg', 'B02_min', 'B02_max', 'B12', 'B11', 'B09', 'B8A', 'B08',
       'B07', 'B06', 'B05', 'B04', 'B03', 'B02', 'B04_avg', 'B05_min',
       'B12_avg', 'B05_avg', 'B12_min', 'B12_max', 'B11_avg', 'B11_min',
       'B11_max', 'B09_avg', 'B09_min', 'B8A_avg', 'B8A_min', 'B8A_max',
       'B08_avg', 'B08_min', 'B08_max', 'B07_avg', 'B07_min', '

In [20]:
features_to_drop = low_importance

In [21]:
X_train_curated = X_train.drop(columns = features_to_drop)
X_test_curated = X_test.drop(columns = features_to_drop)

X_train_curated.shape, X_test_curated.shape

((150582, 77), (49318, 78))

## Cross Validation

In [22]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import BaggingClassifier

cb = CatBoostClassifier(n_estimators = 1500, learning_rate=0.03, depth = 6, random_state = 11, bagging_temperature = 1, task_type = "GPU")

# Use "class_weights = label_weights" for cross validation
cb2 = CatBoostClassifier(n_estimators = 1100, learning_rate=0.03, depth = 6, random_state = 11, bagging_temperature = 1, task_type = "GPU", class_weights = label_weights)

lda = LinearDiscriminantAnalysis()
bc = BaggingClassifier(base_estimator = lda, n_estimators = 30, random_state = 0)
      

In [23]:
# Catboost without weights
cv1 = cross_val_predict(cb, X_train_curated, y_train, cv = 5, method = "predict_proba", verbose = 5)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 2.4170228	total: 27.7ms	remaining: 41.5s
1:	learn: 2.3014428	total: 46.1ms	remaining: 34.5s
2:	learn: 2.2054591	total: 66.7ms	remaining: 33.3s
3:	learn: 2.1210968	total: 85.2ms	remaining: 31.9s
4:	learn: 2.0473640	total: 104ms	remaining: 31.2s
5:	learn: 1.9814927	total: 124ms	remaining: 30.9s
6:	learn: 1.9220395	total: 143ms	remaining: 30.4s
7:	learn: 1.8690365	total: 161ms	remaining: 30.1s
8:	learn: 1.8206386	total: 181ms	remaining: 30s
9:	learn: 1.7768025	total: 204ms	remaining: 30.5s
10:	learn: 1.7359443	total: 223ms	remaining: 30.2s
11:	learn: 1.6987568	total: 242ms	remaining: 30s
12:	learn: 1.6643374	total: 263ms	remaining: 30s
13:	learn: 1.6323075	total: 282ms	remaining: 30s
14:	learn: 1.6021674	total: 302ms	remaining: 29.9s
15:	learn: 1.5738026	total: 319ms	remaining: 29.6s
16:	learn: 1.5461876	total: 339ms	remaining: 29.6s
17:	learn: 1.5206331	total: 357ms	remaining: 29.4s
18:	learn: 1.4968239	total: 375ms	remaining: 29.2s
19:	learn: 1.4742626	total: 392ms	remaining: 

In [ ]:
# Catboost with weights
cv2 = cross_val_predict(cb2, X_train_curated, y_train, cv = 5, method = "predict_proba", verbose = 5)

In [ ]:
# Bagged LDA
cv3 = cross_val_predict(bc, X_train_curated, y_train, cv = 5, method = "predict_proba", verbose = 5)

## Dertermining weights

## Determine weights

In [ ]:
def determine_weights(cvs, y):
    # Code to determine weights
    scores = []

    for w in range(0, 101):
        w = w / 100.
        #  two cross-validation results of choice should be imputed to determine appropriate weights
        scores.append(log_loss(y, (w * cvs[0]) + ((1 - w) * cvs[1])))

    best_score = min(scores)
    weight = scores.index(best_score) / 100.

    return (weight, best_score)

In [ ]:
determine_weights([cv1, cv2], y_train)

In [ ]:
determine_weights([cv1, cv3], y_train)

In [ ]:
determine_weights([cv2, cv3], y_train)

In [ ]:
## Training